## Merge and Scrape


In [47]:
import os
import pandas as pd


def merge_checkpoint_files(directory):
    """Merges _cont-755.csv files with their corresponding _checkpoint.csv files."""
    files = os.listdir(directory)

    file_groups = {}
    for file in files:
        if file.endswith(".csv"):
            base_name = file.replace("_checkpoint.csv", "").replace(
                "_checkpoint_last195.csv", "")
            if base_name not in file_groups:
                file_groups[base_name] = {}
            if "195" in file:
                file_groups[base_name]["cont"] = file
            else:
                file_groups[base_name]["merged"] = file
    print(file_groups)

    for base_name, group in file_groups.items():
        if "cont" in group and "merged" in group:
            merged_path = os.path.join(directory, group["merged"])
            cont_path = os.path.join(directory, group["cont"])

            df_merged = pd.read_csv(merged_path)
            df_cont = pd.read_csv(cont_path)

            merged_df = pd.concat([df_merged, df_cont], ignore_index=True)
            merged_path = os.path.join(
                directory, f"{base_name}_checkpoint.csv")

            merged_df.to_csv(merged_path, index=False)
            print(
                f"Merged {group['merged']} and {group['cont']} -> {merged_path}")


merge_checkpoint_files("checkpoints")

{'pass types': {'merged': 'pass types_checkpoint.csv', 'cont': 'pass types_checkpoint_last195.csv'}, 'possession': {'merged': 'possession_checkpoint.csv', 'cont': 'possession_checkpoint_last195.csv'}, 'goalkeeping': {'merged': 'goalkeeping_checkpoint.csv', 'cont': 'goalkeeping_checkpoint_last195.csv'}, 'passing': {'cont': 'passing_checkpoint_last195.csv', 'merged': 'passing_checkpoint.csv'}, 'shooting': {'merged': 'shooting_checkpoint.csv', 'cont': 'shooting_checkpoint_last195.csv'}, 'miscellaneous stats': {'merged': 'miscellaneous stats_checkpoint.csv', 'cont': 'miscellaneous stats_checkpoint_last195.csv'}, 'advanced goalkeeping': {'merged': 'advanced goalkeeping_checkpoint.csv', 'cont': 'advanced goalkeeping_checkpoint_last195.csv'}, 'goal and shot creation': {'merged': 'goal and shot creation_checkpoint.csv', 'cont': 'goal and shot creation_checkpoint_last195.csv'}, 'defensive actions': {'cont': 'defensive actions_checkpoint_last195.csv', 'merged': 'defensive actions_checkpoint.csv'

# Player SCRAPER


In [1]:
from io import StringIO
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import warnings
import os
import glob

warnings.filterwarnings('ignore')

stats_list = ('standard', 'goalkeeping', 'advanced goalkeeping', 'shooting', 'passing', 'pass types',
              'goal and shot creation', 'defensive actions', 'possession', 'playing time', 'miscellaneous stats')


def available_stats():
    """ Returns the available stats to scrape """
    return stats_list


def get_player_stats_from_URL(url: str, stat: str):
    """ Get player stats from FBref.com, using the given URL
    url: the url to get the stats from
    stat: the stat to get, must be one of the available stats

    returns: pandas dataframe of the stats
    """
    if stat not in stats_list:
        raise ValueError(f'stat must be one of {stats_list}')

    table, row = _get_table_from_URL(url, stat)
    df = _get_dataframe(table, row, url.split("/")[-1])
    return df


def _get_table_from_URL(url, stat):
    # print(f'Getting data from {url}...')
    res = requests.get(url, timeout=10)
    comm = re.compile('<!--|-->')
    soup = bs(comm.sub('', res.text))
    table = soup.find('div', {'id': f'all_stats_{stat}'}).find("table")
    tbod = table.find("tbody").find_all("tr")
    # print(len(tbod))
    # print('Done.')
    # print(table)
    return table.prettify(), len(tbod)


def get_all_player_stats_from_URL(url: str):
    """ Get player stats from FBref.com, using the given URL
    url: the url to get the stats fromKnowing ki you are

    returns: pandas dataframe of the stats
    """
    tables = _gel_all_tables_from_URL(url)
    if tables == -1:
        return -1
    dfs = {}
    for table in tables:
        if table.caption.text.lower().split(":")[0] not in stats_list:
            continue
        df = _get_dataframe(table.prettify(), len(
            table.find("tbody").find_all("tr")), url.split("/")[-1])
        dfs[table.caption.text.lower().split(":")[0]] = df
    # print(f"Processed {url.split("/")[-1]}.")
    return dfs


def _gel_all_tables_from_URL(url):
    try:
        res = requests.get(url, timeout=20)
    except requests.ReadTimeout as e:
        print(f"ReadTimeout: {e}")
        time.sleep(10)
        return _gel_all_tables_from_URL(url)
    comm = re.compile('<!--|-->')
    soup = bs(comm.sub('', res.text))
    tables = soup.find_all('table', {'class': 'stats_table'})
    if not tables:
        print("No tables found for {}. Got error - {}".format(url, res.status_code))
        return -1
    return tables


def _get_dataframe(table, row, name):
    df = pd.read_html(StringIO(table))
    df = df[0]

    # delete the last column (Rk, Match)
    df = df.iloc[:, :-1]

    # keep only the second value for the headers

    df.columns = [h[1] for h in df.columns]

    # only keep the part after space for 'Nation'
    df['Country'] = df['Country'].apply(
        lambda x: str(x).rsplit(' ', maxsplit=1)[-1])

    # delete rows with the column names
    df = df[df[df.columns[0]] != df.columns[0]]
    df.reset_index(drop=True, inplace=True)
    df.drop(["Age", "Squad", "Country", "Comp", "LgRank"], axis=1, inplace=True)

    # convert all numeric columns to numeric
    df = df.apply(pd.to_numeric, errors='ignore')
    df = df.iloc[[row]]
    df.insert(0, "Player Name", name)

    return df

In [2]:
merged_df = pd.read_csv("../data/urls/merged_urls.csv")
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.reset_index(drop=True)
print(merged_df.shape)

(2596, 2)


## Scrape Player data


In [5]:
from tqdm import tqdm

CHECKPOINT_INTERVAL = 50

stats_dict = {stat: None for stat in stats_list}
fdf = merged_df["urls"][-196:]

for i, url in enumerate(tqdm(fdf), start=1):
    dfs = get_all_player_stats_from_URL(f"https://fbref.com{url}")
    while dfs == -1:
        print("Error 429... Breaking")
        time.sleep(10*60)
        dfs = get_all_player_stats_from_URL(f"https://fbref.com{url}")
    for stat in list(dfs.keys()):
        if stats_dict[stat] is None:
            stats_dict[stat] = dfs[stat]
        else:
            stats_dict[stat] = pd.concat([stats_dict[stat], dfs[stat]], axis=0)

    # print(url.split("/")[-1], " : ",stats_dict)

    if i % CHECKPOINT_INTERVAL == 0:
        print(f"Checkpoint reached: Processed {i} rows. Saving progress...")
        os.makedirs("checkpoints", exist_ok=True)
        for stat, df in stats_dict.items():
            if df is not None:
                # print(df)
                df.to_csv(
                    f"checkpoints/{stat}_checkpoint.csv", index=False)
                # print(f"Checkpoint saved for {stat}!")
        print("Checkpoint saved!")
        time.sleep(9)

    time.sleep(3)

NameError: name 'merged_df' is not defined

In [ ]:
# @title remove duplicates and store them with same name

for file in glob.glob("checkpoints/*.csv"):
    df = pd.read_csv(file)
    df.drop_duplicates(inplace=True)
    df.to_csv(file, index=False)

### Change positions in player data


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("merged-tm-fb.csv")

df.loc[df['position'] == 'Second Striker', 'position'] = 'Attacking Midfield'
df.loc[df['position'] == 'Left Midfield', 'position'] = 'Left Winger'
df.loc[df['position'] == 'Right Midfield', 'position'] = 'Right Winger'

positions = df['position'].unique()
for pos in positions:
    # Create a DataFrame for each position
    df_pos = df[df['position'] == pos]

    file_name = f"player_data_{pos}.csv"
    df_pos.to_csv(f"../data/position data/24-25/{file_name}", index=False)

print("DataFrames saved successfully!")

DataFrames saved successfully!


### Add league to player data


In [ ]:
# @title Add league to player data

import pandas as pd

# leagues
premier = pd.read_csv("../data/urls-Premier League.csv")
laliga = pd.read_csv("../data/urls-La Liga.csv")
seriea = pd.read_csv("../data/urls-Serie A.csv")
bundesliga = pd.read_csv("../data/urls-Bundesliga.csv")
ligue1 = pd.read_csv("../data/urls-Ligue 1.csv")

# merged df
df = pd.read_csv("../data/merged-tm-fb.csv")

# Create a dictionary to map player URLs to league names
league_dict = {}

for league_df, league_name in zip([premier, laliga, seriea, bundesliga, ligue1],
                                  ['Premier League', 'La Liga', 'Serie A', 'Bundesliga', 'Ligue 1']):
    for url in league_df['urls']:
        league_dict[url.split("/")[-1].replace("-", " ")] = league_name

# Add the league column to df
df['league'] = df['name'].map(league_dict)
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

df.to_csv("../data/merged-tm-fb.csv", index=False)

### Merge all stats data


In [26]:
# @title: Combine columns and rename cols with same name different values

import pandas as pd
import glob

csv_files = glob.glob("../data/season wise data/2024-2025/*.csv")

dfs = []
cols = []
fdf = pd.DataFrame()
skip = ["Player", "Nation", "Pos", "Squad", "Comp", "Age", "Born", "MP", "Starts", "Min", "90s", "Gls", "Ast", "G+A", "G-PK", "PK", "PKatt", "CrdY",
        "CrdR", "xG", "npxG", "xAG", "npxG+xAG", "PrgC", "PrgP", "PrgR", "Gls", "Ast",  "G+A", "G-PK", "G+A-PK", "xG", "xAG", "xG+xAG", "npxG", "npxG+xAG"]

for file in csv_files:
    df = pd.read_csv(file)
    df.drop("Unnamed: 0", axis=1, inplace=True)
    for col in df.columns:
        if col in skip:
            continue
        if col in cols:
            if df[col].equals(fdf[col]):
                continue
            else:
                df.rename(
                    columns={col: f"{col}_{file.split('/')[-1].split('.')[0]}"}, inplace=True)
        cols.append(col)
    dfs.append(df)
    fdf = pd.concat([fdf, df], axis=0, ignore_index=True)

final_df = pd.concat(dfs, axis=0, ignore_index=True)

final_df = final_df.groupby("Player", as_index=False).first()

final_df.to_csv("combined_players_data.csv", index=False)

print("CSV files successfully combined!")

CSV files successfully combined!


In [29]:
# cpdf = pd.read_csv("combined_players_data.csv")
# tmdf = pd.read_csv("../data/tm data/merged_tm.csv")
# cpdf = cpdf.rename(columns = {"Player": "name"})
# cpdf["name"] = cpdf.name.apply(lambda x: x.replace("-", " "))
# tmdf = tmdf.drop('Unnamed: 0', axis = 1)
# df = cpdf.merge(tmdf, on="name", how="inner")
# duplicate_columns = df.columns[df.columns.duplicated()].tolist()
# print("Duplicate Columns:", duplicate_columns)
# dfdrop = df.T.drop_duplicates().T
# df.to_csv("merged-tm-fb.csv")

Duplicate Columns: []


# TEAM SCRAPER


In [5]:
from io import StringIO
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from bs4 import NavigableString
import time
import warnings
import os
import glob

warnings.filterwarnings('ignore')

stats_list = {'standard': 'standard', 'keepers': 'keeper', 'keepersadv': "keeper_adv", 'shooting': "shooting", 'passing': "passing", 'passing_types': "passing_types",
              'gca': "gca", 'defense': "defense", 'possession': "possession", 'playingtime': "playing_time", 'misc': "misc"}


class TeamScraper():
    def __init__(self, url):
        self.url = url

    def get_dfs(self):
        self.dfs = self.get_all_team_stats_from_URL(self.url)
        return self.dfs

    def get_all_team_stats_from_URL(self, url: str):
        """ Get player stats from FBref.com, using the given URL
        url: the url to get the stats fromKnowing ki you are

        returns: pandas dataframe of the stats
        """
        tables = self._gel_all_team_tables(url)
        if tables == -1:
            return -1
        dfs = {}
        for table in tables:
            # if table.caption.text.lower().split(":")[0] not in stats_list:
            #     continue
            df = self._get_dataframe(table.prettify())
            dfs[table.caption.text.lower().split(":")[0]] = df
        # print(f"Processed {url.split("/")[-1]}.")
        return dfs

    def _gel_all_team_tables(self, url):
        try:
            res = requests.get(url, timeout=20)
        except requests.ReadTimeout as e:
            print(f"ReadTimeout: {e}")
            time.sleep(10)
            return self._gel_all_team_tables(url)
        # comm = re.compile('<!--|-->')
        # self.soup = bs(comm.sub('', res.text))
        self.soup = bs(res.text)
        tables = []
        for div in self.soup.find_all('div', id=lambda x: x and 'for' in x):
            table = div.find('table', class_='stats_table')
            if table:
                tables.append(table)
        if not tables:
            print("No tables found for {}. Got error - {}".format(url, res.status_code))
            return -1
        return tables

    def _get_dataframe(self, table):
        df = pd.read_html(StringIO(table))
        df = df[0]

        # delete the last column (Rk, Match)
        df = df.iloc[:, :-1]

        # keep only the second value for the headers
        # print(df.columns)
        df.columns = [h[0] for h in df.columns]

        # delete rows with the column names
        # df = df[df[df.columns[0]] != df.columns[0]]
        df.reset_index(drop=True, inplace=True)
        df.drop(["Age", "Squad", "Country", "Comp", "LgRank"],
                axis=1, inplace=True, errors='ignore')

        # convert all numeric columns to numeric
        df = df.apply(pd.to_numeric, errors='ignore')

        return df

    def get_league_leaders(self):
        res = requests.get(self.url, timeout=20)
        comm = re.compile('<!--|-->')
        soup = bs(comm.sub('', res.text))
        leaders = soup.find('div', {'id': 'div_leaders'})
        if not leaders:
            print("No leaders found for {}. Got error".format(self.url))
            return -1
        leaders_div = leaders.find_all('table', {'class': 'columns'})
        names = [leader.caption.text for leader in leaders_div]
        # print(leaders_div)
        leaders = [leader.prettify()
                   for leader in leaders if not isinstance(leader, NavigableString)]
        leaders = [pd.read_html(StringIO(leader)) for leader in leaders]
        leaders = [leader[0].rename(columns={leader[0].columns[0]: "Rank", leader[0].columns[1]
                                    : "Player", leader[0].columns[2]: "Value"}) for leader in leaders]
        return {names[i]: leaders[i] for i in range(len(names))}

### Scrape teams data


In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm

URL = "https://fbref.com/en/comps/{}/{}-Stats"

LEAGUES = {
    "Premier League": 9,
    "La Liga": 12,
    "Bundesliga": 20,
    "Serie A": 11,
    "Ligue 1": 13,
}

for k, v in LEAGUES.items():
    print(f"League: {k}")
    print(f"URL: {URL.format(v, k.replace(' ', '-'))}")
    print()
    ts = TeamScraper(URL.format(v, k.replace(' ', '-')))
    dfs = ts.get_dfs()
    for key, value in dfs.items():
        if value is not None:
            # if not os.path.exists(f"../data/team data/{k}/{key}.csv"):
            os.makedirs(f"../data/team data/{k}", exist_ok=True)
            value.to_csv(
                f"../data/team data/{k}/{key}.csv", index=False)
            print(f"df saved for {key}!")
    leaders = ts.get_league_leaders()
    os.makedirs(f"../data/team data/{k}/Leaders/", exist_ok=True)
    for name, leader_df in leaders.items():
        if leader_df is not None:
            try:
                leader_df.to_csv(
                    f"../data/team data/{k}/Leaders/{name.replace("/", "-")}.csv", index=False)
            except:
                print(f"Error saving {name}...")
                continue
        else:
            print(f"No data for {name}...")

    print(f"leaders saved for {name}!")

League: Premier League
URL: https://fbref.com/en/comps/9/Premier-League-Stats

df saved for squad standard stats 2024-2025 premier league table!
df saved for squad goalkeeping 2024-2025 premier league table!
df saved for squad advanced goalkeeping 2024-2025 premier league table!
df saved for squad shooting 2024-2025 premier league table!
df saved for squad passing 2024-2025 premier league table!
df saved for squad pass types 2024-2025 premier league table!
df saved for squad goal and shot creation 2024-2025 premier league table!
df saved for squad defensive actions 2024-2025 premier league table!
df saved for squad possession 2024-2025 premier league table!
df saved for squad playing time 2024-2025 premier league table!
df saved for squad miscellaneous stats 2024-2025 premier league table!
leaders saved for PSxG-GA/90!
League: La Liga
URL: https://fbref.com/en/comps/12/La-Liga-Stats

df saved for squad standard stats 2024-2025 la liga table!
df saved for squad goalkeeping 2024-2025 la 